d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Querying JSON & Hierarchical Data with SQL

Apache Spark&trade; and Databricks&reg; make it easy to work with hierarchical data, such as nested JSON records.

## In this lesson you:
* Use SQL to query a table backed by JSON data
* Query nested structured data
* Query data containing array columns 

## Audience
* Primary Audience: Data Analysts
* Additional Audiences: Data Engineers and Data Scientists

## Prerequisites
* Web browser: **Chrome**
* A cluster configured with **8 cores** and **DBR 6.3**
* Familiarity with <a href="https://www.w3schools.com/sql/" target="_blank">ANSI SQL</a> is required

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Setup & Classroom-Cleanup<br>

For each lesson to execute correctly, please make sure to run the **`Classroom-Setup`** cell at the start of each lesson (see the next cell) and the **`Classroom-Cleanup`** cell at the end of each lesson.

In [4]:
%run "./Includes/Classroom-Setup"

<iframe  
src="//fast.wistia.net/embed/iframe/a3098jg2t0?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/a3098jg2t0?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

## Examining the contents of a JSON file

JSON is a common file format in big data applications and in data lakes (or large stores of diverse data).  Datatypes such as JSON arise out of a number of data needs.  For instance, what if...  
<br>
* Your schema, or the structure of your data, changes over time?
* You need nested fields like an array with many values or an array of arrays?
* You don't know how you're going use your data yet so you don't want to spend time creating relational tables?

The popularity of JSON is largely due to the fact that JSON allows for nested, flexible schemas.

This lesson uses the `DatabricksBlog` table, which is backed by JSON file `dbfs:/mnt/training/databricks-blog.json`. If you examine the raw file, you can see that it contains compact JSON data. There's a single JSON object on each line of the file; each object corresponds to a row in the table. Each row represents a blog post on the <a href="https://databricks.com/blog" target="_blank">Databricks blog</a>, and the table contains all blog posts through August 9, 2017.

<iframe  
src="//fast.wistia.net/embed/iframe/1i3n3rb0vy?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/1i3n3rb0vy?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

In [8]:
%fs head dbfs:/mnt/training/databricks-blog.json

To expose the JSON file as a table, use the standard SQL create table using syntax introduced in the previous lesson:

In [10]:
%sql
CREATE TABLE IF NOT EXISTS DatabricksBlog
  USING json
  OPTIONS (
    path "dbfs:/mnt/training/databricks-blog.json",
    inferSchema "true"
  )

Take a look at the schema with the `DESCRIBE` function.

In [12]:
%sql
DESCRIBE DatabricksBlog

Run a query to view the contents of the table.

Notice:
* The `authors` column is an array containing multiple author names.
* The `categories` column is an array of multiple blog post category names.
* The `dates` column contains nested fields `createdOn`, `publishedOn` and `tz`.

In [14]:
%sql
SELECT authors, categories, dates, content 
FROM DatabricksBlog

## Nested Data

Think of nested data as columns within columns. 

For instance, look at the `dates` column.

<iframe  
src="//fast.wistia.net/embed/iframe/kqmfblujy9?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/kqmfblujy9?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

In [17]:
%sql
SELECT dates FROM DatabricksBlog

Pull out a specific subfield with "dot" notation.

In [19]:
%sql
SELECT dates.createdOn, dates.publishedOn 
FROM DatabricksBlog

Both `createdOn` and `publishedOn` are stored as strings.

Cast those values to SQL timestamps:

In this case, use a single `SELECT` statement to:
0. Cast `dates.publishedOn` to a `timestamp` data type.
0. "Flatten" the `dates.publishedOn` column to just `publishedOn`.

In [21]:
%sql
SELECT title, 
       cast(dates.publishedOn AS timestamp) AS publishedOn 
FROM DatabricksBlog

Create the temporary view `DatabricksBlog2` to capture the conversion and flattening of the `publishedOn` column.

In [23]:
%sql
CREATE OR REPLACE TEMPORARY VIEW DatabricksBlog2 AS
  SELECT *, 
         cast(dates.publishedOn AS timestamp) AS publishedOn 
  FROM DatabricksBlog

Now that we have this temporary view, we can use `DESCRIBE` to check its schema and confirm the timestamp conversion.

In [25]:
%sql
DESCRIBE DatabricksBlog2

-sandbox
Now the dates are represented by a `timestamp` data type, query for articles within certain date ranges (such as getting a list of all articles published in 2013), and format the date for presentation purposes.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> See the Spark documentation, <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions" target="_blank">built-in functions</a>, for a long list of date-specific functions.

In [27]:
%sql
SELECT title, 
       date_format(publishedOn, "MMM dd, yyyy") AS date, 
       link 
FROM DatabricksBlog2
WHERE year(publishedOn) = 2013
ORDER BY publishedOn

## Array Data

The table also contains array columns. 

Easily determine the size of each array using the built-in `size(..)` function with array columns.

<iframe  
src="//fast.wistia.net/embed/iframe/w9vj8mjpf7?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/w9vj8mjpf7?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

In [30]:
%sql
SELECT size(authors), 
       authors 
FROM DatabricksBlog

Pull the first element from the array `authors` using an array subscript operator.

In [32]:
%sql
SELECT authors[0] AS primaryAuthor 
FROM DatabricksBlog

### Explode

The `explode` function allows you to split an array column into multiple rows, copying all the other columns into each new row. 

For example, you can split the column `authors` into the column `author`, with one author per row.

<iframe  
src="//fast.wistia.net/embed/iframe/h8tv263d04?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/h8tv263d04?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

In [35]:
%sql
SELECT title, 
       authors, 
       explode(authors) AS author, 
       link 
FROM DatabricksBlog

It's more obvious to restrict the output to articles that have multiple authors, and sort by the title.

In [37]:
%sql
SELECT title, 
       authors, 
       explode(authors) AS author, 
       link 
FROM DatabricksBlog 
WHERE size(authors) > 1 
ORDER BY title

### Lateral View
The data has multiple columns with nested objects.  In this case, the data has multiple dates, authors, and categories.

Take a look at the blog entry **Apache Spark 1.1: The State of Spark Streaming**:

In [39]:
%sql
SELECT dates.publishedOn, title, authors, categories
FROM DatabricksBlog
WHERE title = "Apache Spark 1.1: The State of Spark Streaming"

Next, use `LATERAL VIEW` to explode multiple columns at once, in this case, the columns `authors` and `categories`.

In [41]:
%sql
SELECT dates.publishedOn, title, author, category
FROM DatabricksBlog
LATERAL VIEW explode(authors) exploded_authors_view AS author
LATERAL VIEW explode(categories) exploded_categories AS category
WHERE title = "Apache Spark 1.1: The State of Spark Streaming"
ORDER BY author, category

## Exercise 1

Identify all the articles written or co-written by Michael Armbrust.

-sandbox
### Step 1

Starting with the table `DatabricksBlog`, create a temporary view called `ArticlesByMichael` where:
0. Michael Armbrust is the author
0. The data set contains the column `title` (it may contain others)
0. It contains only one record per article

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** See the Spark documentation, <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions" target="_blank">built-in functions</a>.  

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** Include the column `authors` in your view, to help you debug your solution.

In [44]:
%sql
-- TODO

FILL_IN

In [45]:
# TEST - Run this cell to test your solution.

resultsDF = spark.sql("select title from ArticlesByMichael order by title")
dbTest("SQL-L5-articlesByMichael-count", 3, resultsDF.count())

results = [r[0] for r in resultsDF.collect()]
dbTest("SQL-L5-articlesByMichael-0", "Exciting Performance Improvements on the Horizon for Spark SQL", results[0])
dbTest("SQL-L5-articlesByMichael-1", "Spark SQL Data Sources API: Unified Data Access for the Apache Spark Platform", results[1])
dbTest("SQL-L5-articlesByMichael-2", "Spark SQL: Manipulating Structured Data Using Apache Spark", results[2])

print("Tests passed!")

### Step 2
Show the list of Michael Armbrust's articles.

In [47]:
%sql
-- TODO

FILL_IN

## Exercise 2

Identify the complete set of categories used in the Databricks blog articles.

### Step 1

Starting with the table `DatabricksBlog`, create another view called `UniqueCategories` where:
0. The data set contains the one column `category` (and no others)
0. This list of categories should be unique

In [50]:
%sql
-- TODO

FILL_IN

In [51]:
# TEST - Run this cell to test your solution.

resultsCount = spark.sql("SELECT category FROM UniqueCategories order by category")

dbTest("SQL-L5-uniqueCategories-count", 12, resultsCount.count())

results = [r[0] for r in resultsCount.collect()]
dbTest("SQL-L5-uniqueCategories-0", "Announcements", results[0])
dbTest("SQL-L5-uniqueCategories-1", "Apache Spark", results[1])
dbTest("SQL-L5-uniqueCategories-2", "Company Blog", results[2])

dbTest("SQL-L5-uniqueCategories-9", "Platform", results[9])
dbTest("SQL-L5-uniqueCategories-10", "Product", results[10])
dbTest("SQL-L5-uniqueCategories-11", "Streaming", results[11])

print("Tests passed!")

### Step 2
Show the complete list of categories.

In [53]:
%sql
-- TODO

FILL_IN

## Exercise 3

Count how many times each category is referenced in the Databricks blog.

-sandbox
### Step 1

Starting with the table `DatabricksBlog`, create a temporary view called `TotalArticlesByCategory` where:
0. The new table contains two columns, `category` and `total`
0. The `category` column is a single, distinct category (similar to the last exercise)
0. The `total` column is the total number of articles in that category

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** You need either multiple views or a `LATERAL VIEW` to solve this.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Because articles can be tagged with multiple categories, the sum of the totals adds up to more than the total number of articles.

In [56]:
%sql
-- TODO

FILL_IN

In [57]:
# TEST - Run this cell to test your solution.

resultsDF = spark.sql("SELECT category, total FROM TotalArticlesByCategory ORDER BY category")
dbTest("SQL-L5-articlesByCategory-count", 12, resultsDF.count())

results = [ (r[0]+" w/"+str(r[1])) for r in resultsDF.collect()]

dbTest("SQL-L5-articlesByCategory-0", "Announcements w/72", results[0])
dbTest("SQL-L5-articlesByCategory-1", "Apache Spark w/132", results[1])
dbTest("SQL-L5-articlesByCategory-2", "Company Blog w/224", results[2])

dbTest("SQL-L5-articlesByCategory-9", "Platform w/4", results[9])
dbTest("SQL-L5-articlesByCategory-10", "Product w/83", results[10])
dbTest("SQL-L5-articlesByCategory-11", "Streaming w/21", results[11])

print("Tests passed!")

### Step 2
Display the totals of each category, order by `category`.

In [59]:
%sql
-- TODO

FILL_IN

## Summary

* Spark SQL allows you to query and manipulate structured and semi-structured data
* Spark SQL's built-in functions provide powerful primitives for querying complex schemas

## Review Questions
**Q:** What is the syntax for accessing nested columns?  
**A:** Use the dot notation: ```SELECT dates.publishedOn```

**Q:** What is the syntax for accessing the first element in an array?  
**A:** Use the [subscript] notation:  ```SELECT authors[0]```

**Q:** What is the syntax for expanding an array into multiple rows?  
**A:** Use the explode keyword, either:  
```SELECT explode(authors) as Author``` or  
```LATERAL VIEW explode(authors) exploded_authors_view AS author```

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Cleanup<br>

Run the **`Classroom-Cleanup`** cell below to remove any artifacts created by this lesson.

In [63]:
%run "./Includes/Classroom-Cleanup"

## Next Steps

Start the next lesson, [Querying Data Lakes with SQL]($./SSQL 06 - Data Lakes).

## Additional Topics & Resources

* <a href="https://docs.databricks.com/spark/latest/spark-sql/index.html" target="_blank">Spark SQL Reference</a>
* <a href="http://spark.apache.org/docs/latest/sql-programming-guide.html" target="_blank">Spark SQL, DataFrames and Datasets Guide</a>
* <a href="https://stackoverflow.com/questions/36876959/sparksql-can-i-explode-two-different-variables-in-the-same-query" target="_blank">SparkSQL: Can I explode two different variables in the same query? (StackOverflow)</a>

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>